In [1]:
import tensorflow as tf
import os

# =====================
# PATH
# =====================
MODEL_PATH = r"C:\Users\adity\Downloads/Code/Knowledge_Distillation/baseline_mobilenetv3_final.keras"
OUTPUT_TFLITE = r"C:\Users\adity\Downloads/Code/Quantization/testdataluar_fp16.tflite"

# =====================
# LOAD MODEL
# =====================
model = tf.keras.models.load_model(
    MODEL_PATH,
    compile=False
)

print("✅ Baseline FP32 model loaded")


✅ Baseline FP32 model loaded


In [2]:
# =====================
# FP16 QUANTIZATION
# =====================
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_fp16 = converter.convert()

# =====================
# SAVE MODEL
# =====================
with open(OUTPUT_TFLITE, "wb") as f:
    f.write(tflite_fp16)

print("✅ FP16 TFLite model saved:", OUTPUT_TFLITE)
print("📦 Model size (MB):", round(os.path.getsize(OUTPUT_TFLITE) / (1024**2), 2))


INFO:tensorflow:Assets written to: C:\Users\adity\AppData\Local\Temp\tmpct776edq\assets


INFO:tensorflow:Assets written to: C:\Users\adity\AppData\Local\Temp\tmpct776edq\assets


Saved artifact at 'C:\Users\adity\AppData\Local\Temp\tmpct776edq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  1511937154448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1511937155024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1511937155600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1511937155984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1511937155216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1511937154256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1511937157904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1511937156752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1511937157136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1511937156560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  151

In [3]:
# =====================
# DISABLE XNNPACK (WAJIB DI WINDOWS)
# =====================
os.environ["TF_LITE_DISABLE_XNNPACK"] = "1"

# =====================
# LOAD INTERPRETER
# =====================
interpreter = tf.lite.Interpreter(
    model_path=OUTPUT_TFLITE
)
interpreter.allocate_tensors()

input_details  = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("✅ FP16 Interpreter ready")
print("Input :", input_details[0]["shape"], input_details[0]["dtype"])
print("Output:", output_details[0]["shape"], output_details[0]["dtype"])


✅ FP16 Interpreter ready
Input : [  1 224 224   3] <class 'numpy.float32'>
Output: [ 1 10] <class 'numpy.float32'>


C:\Users\adity\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [4]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input

# =====================
# CONFIG
# =====================
IMG_SIZE = 224
BATCH_SIZE = 16
TEST_DIR =  r"C:\Users\adity\Downloads/Tomato Leaf Disease Dataset/TomatoDataset"

# =====================
# TEST GENERATOR (SAMA DENGAN TRAINING)
# =====================
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

print("✅ test_gen ready")
print("Total test samples:", test_gen.samples)


Found 1609 images belonging to 10 classes.
✅ test_gen ready
Total test samples: 1609


In [5]:
import time
import numpy as np

correct = 0
total = 0
times = []

for i in range(test_gen.samples):
    x_batch, y_batch = next(test_gen)
    x = x_batch[0:1]
    y = y_batch[0]

    start = time.time()
    interpreter.set_tensor(input_details[0]["index"], x.astype(np.float32))
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]["index"])
    end = time.time()

    pred = output.argmax(axis=1)[0]
    true = y.argmax()

    correct += int(pred == true)
    total += 1
    times.append(end - start)

accuracy = correct / total
avg_time = sum(times) / len(times)
fps = 1 / avg_time

print(f"✅ TFLite FP16 Accuracy : {accuracy*100:.2f}%")
print(f"⏱ Avg inference / img  : {avg_time:.6f} sec")
print(f"🚀 FPS                 : {fps:.2f}")


✅ TFLite FP16 Accuracy : 46.74%
⏱ Avg inference / img  : 0.003885 sec
🚀 FPS                 : 257.39
